<a href="https://colab.research.google.com/github/leoxiang66/Knowledge-driven-spoken-dialogue/blob/main/%E8%8E%B7%E5%BE%97%E5%AF%B9%E8%AF%9D%E7%9A%84%E8%AE%AD%E7%BB%83%E6%95%B0%E6%8D%AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets

In [2]:
!git lfs install
!git clone https://huggingface.co/datasets/Adapting/2022-GLOBAL-AI-CHALLENGE

Error: Failed to call git rev-parse --git-dir --show-toplevel: "fatal: not a git repository (or any of the parent directories): .git\n"
Git LFS initialized.
Cloning into '2022-GLOBAL-AI-CHALLENGE'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 29 (delta 12), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (29/29), done.


In [3]:
%ls 2022-GLOBAL-AI*

kg.json  knowledge_data.csv  test.json  train.json  valid.json


Load knowledge graph data

In [3]:
data_path = '2022-GLOBAL-AI-CHALLENGE'

In [4]:
import pandas as pd

klg_data = pd.read_csv(f'{data_path}/knowledge_data.csv')
klg_data

,entity,knowledge
0,调王生,[ett] 的时间是明朝。[ett] 的作者是袁凯。[ett] 的诗句是门外桃花落渐多，一双...
1,载伎重游王潭马砦岩,[ett] 的时间是明朝。[ett] 的作者是程嘉燧。[ett] 的诗句是疏松历落映层台，绿...
2,观军装十咏·马,[ett] 的时间是明朝。[ett] 的作者是高启。[ett] 的诗句是罗帕覆春风，连鞍赐有...
3,菩萨蛮 回文 中州乐府,[ett] 的时间是元朝。[ett] 的作者是孟宗献。[ett] 的诗句是睡惊秋近鸣虫砌。砌...
4,西江月·往事俄惊如梦,[ett] 的时间是元朝。[ett] 的作者是钱应庚。[ett] 的诗句是往事俄惊如梦，白头...
...,...,...
38049,贝恩·血蹄,[ett] 的亲属(先祖)是血蹄长者。[ett] 的亲属(母亲)是塔玛拉。[ett] 的亲属...
38050,怨恨回响,[ett] 的所在地是七星殿。[ett] 的种族是煞（元素生物）。[ett] 的等级是90。...
38051,伊芙·费德里克森,[ett] 的势力是提瑞斯法议会。[ett] 的头衔是提瑞斯法议会的回忆。[ett] 的对联...
38052,女伯爵莉亚德琳,[ett] 的亲属(养女)是萨兰蒂亚。[ett] 的亲属(导师/养父)是高阶牧师冯德洛尔。[...


Load train/valid set

In [5]:
import json
with open(f'{data_path}/valid.json', encoding='utf-8') as f:
    data = json.load(f)

print(type(data))

<class 'list'>


### Add entries to the original dataset

In [6]:
import pandas as pd
df = pd.DataFrame(columns=['chat_history','last_usr_uttr','entity','attrs','knowledge','knowledge_index','response'])
df

,chat_history,last_usr_uttr,entity,attrs,knowledge,knowledge_index,response


In [7]:
def construct_entry(sample, i, chat_history, count):
    last_usr_uttr = sample[i]['message']
    chat_history.append(last_usr_uttr)
    attrs = sample[i+1].get('attrs',None)
    response = sample[i+1]['message']
    if attrs is not None:
        entity = attrs[-1]['name']
        cond_match_entity = klg_data['entity']==entity
        knowledge = klg_data.loc[cond_match_entity,'knowledge'].values[0]
        knowledge_index = klg_data.index[cond_match_entity].tolist()[0]
    else:
        entity = None
        knowledge = None
        knowledge_index = None

    df.loc[count,'chat_history'] = copy.deepcopy(chat_history)
    df.loc[count,'attrs'] = attrs
    df.loc[count,'response'] = response
    df.loc[count,'last_usr_uttr'] = last_usr_uttr
    df.loc[count,'entity'] = entity
    df.loc[count,'knowledge'] = knowledge
    df.loc[count,'knowledge_index'] = knowledge_index

    chat_history.append(response)
    count +=1
    
    return count, chat_history

In [8]:
import copy
from tqdm import tqdm
count = 0

for j in tqdm(range(len(data))):
    sample = data[j]['messages']
    chat_history = []

    if len(sample)/2 == 0:
        for i in range(0,len(sample),2):
            count, chat_history = construct_entry(sample, i, chat_history, count)
            
    else:
        for i in range(0,len(sample)-1,2):
            count, chat_history = construct_entry(sample, i, chat_history, count)

100%|████████████████████████████████████████████████████████████████████████████████| 740/740 [00:17<00:00, 42.39it/s]


In [9]:
df

,chat_history,last_usr_uttr,entity,attrs,knowledge,knowledge_index,response
0,[你好，你知道中国科学院植物研究所北京植物园吗呀？],你好，你知道中国科学院植物研究所北京植物园吗呀？,中国科学院植物研究所北京植物园,"[{'attrname': 'Information', 'attrvalue': '中国科...",[ett] 的Information是中国科学院植物研究所北京植物园位于北京市海淀区香山南路...,36902,当然知道了，植物园包括树木园、宿根花卉园、月季园、牡丹园、本草园、紫薇园、野生果树资源区、珍...
1,"[你好，你知道中国科学院植物研究所北京植物园吗呀？, 当然知道了，植物园包括树木园、宿根花卉...",啊，看来很不错啊啊，这里的门票多呢？,中国科学院植物研究所北京植物园,"[{'attrname': '门票', 'attrvalue': '成人票10元。1.2米以...",[ett] 的Information是中国科学院植物研究所北京植物园位于北京市海淀区香山南路...,36902,成人票10元。1.2米以下儿童免票；大、中、小学生（不含成人教育）凭有效证件半价；老年人凭老...
2,"[你好，你知道中国科学院植物研究所北京植物园吗呀？, 当然知道了，植物园包括树木园、宿根花卉...",开放时间是8:00-16:30，你知道具体地址吗？,中国科学院植物研究所北京植物园,"[{'attrname': '地址', 'attrvalue': '北京市海淀区香山南辛村2...",[ett] 的Information是中国科学院植物研究所北京植物园位于北京市海淀区香山南路...,36902,地址是北京市海淀区香山南辛村20号 ‎，近北京植物园。
3,"[你好，你知道中国科学院植物研究所北京植物园吗呀？, 当然知道了，植物园包括树木园、宿根花卉...",这里可以玩几个小时啊恩？,中国科学院植物研究所北京植物园,"[{'attrname': '建议游玩时间', 'attrvalue': '1小时', 'n...",[ett] 的Information是中国科学院植物研究所北京植物园位于北京市海淀区香山南路...,36902,能玩1小时，他周边的景点也很不错的。
4,"[你好，你知道中国科学院植物研究所北京植物园吗呀？, 当然知道了，植物园包括树木园、宿根花卉...",是，我想去旁边的香山公园玩玩。,香山公园,"[{'attrname': '门票', 'attrvalue': '淡季（11月16日-3月...",[ett] 的周边景点是八处证果寺。[ett] 的门票是淡季（11月16日-3月31日）5元...,36529,香山公园票价不贵，淡季（11月16日-3月31日）5元，旺季（4月1日-11月15日）10元...
...,...,...,...,...,...,...,...
6558,"[你好，你知道安不忘危这个成语吗呵？, 我了解过安不忘危这个成语，你想了解什么呢。, 那你能...",那你能说下安不忘危这个成语的含义嘛？,安不忘危,"[{'attrname': '释义', 'attrvalue': '在安全的时候不忘记危难。...",[ett] 的中文是安不忘危。[ett] 的反义词是高枕无忧。[ett] 的近义词是居安思危...,25408,好的，安不忘危形容的是时刻谨慎小心，提高警惕。
6559,"[你好，你知道安不忘危这个成语吗呵？, 我了解过安不忘危这个成语，你想了解什么呢。, 那你能...",这个成语我很中意，你能告诉安不忘。危的出处吗？,安不忘危,"[{'attrname': '出处', 'attrvalue': '《周易·系辞下》是故君子...",[ett] 的中文是安不忘危。[ett] 的反义词是高枕无忧。[ett] 的近义词是居安思危...,25408,这个没问题，当然知道。安不忘危的出处是《周易·系辞下》。
6560,"[你好，你知道安不忘危这个成语吗呵？, 我了解过安不忘危这个成语，你想了解什么呢。, 那你能...",你能用一个故事来说明一下安不忘危成语这个呀吗,安不忘危,"[{'attrname': '故事', 'attrvalue': '我国最早的一部占卜书《周...",[ett] 的中文是安不忘危。[ett] 的反义词是高枕无忧。[ett] 的近义词是居安思危...,25408,可以的，我听过这个故事，安不忘危说的是我国最早的一部占卜书《周易》关于吉凶祸福有深刻的论述，...
6561,"[你好，你知道安不忘危这个成语吗呵？, 我了解过安不忘危这个成语，你想了解什么呢。, 那你能...",安不忘危的拼音是。怎样的呢？,安不忘危,"[{'attrname': '拼音', 'attrvalue': 'ān bù wàng w...",[ett] 的中文是安不忘危。[ett] 的反义词是高枕无忧。[ett] 的近义词是居安思危...,25408,很容易的一个拼音，我告诉你吧，安不忘危的拼音是: ān bù wàng wēi


In [10]:
df.to_csv(f'{data_path}/valid.csv', index=False)